In [1]:
import pandas as pd
import numpy as np
import re

from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, STAMEN_TONER, CARTODBPOSITRON_RETINA
from bokeh.models import HoverTool, FreehandDrawTool, BoxEditTool, ColumnDataSource, ColorBar
from bokeh.palettes import Plasma10
from bokeh.transform import linear_cmap

In [2]:
# Define function to switch from lat/long to mercator x/y coordinates
def to_mercator(lat, lon):
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
geo = pd.read_csv('sectors.csv')
geo.shape

(2624, 7)

In [4]:
data = pd.read_csv('ALL_data.csv', index_col='PID')
data.shape

(2579, 81)

In [5]:
map_data = geo.merge(np.round(data[['SalePrice']]/1000),how='left',on='PID')
map_data.drop_duplicates(inplace=True)

In [6]:
# Ames, Iowa lat long, converted to mercator
Ames_center = to_mercator(42.034534, -93.620369)

In [7]:
landmarks = {'landmarks':['Iowa State University',
                          'Municipal Airport',
                          'North Grand Mall',
                          'Mary Greeley Medical Center',
                          'Jack Trice Stadium'],
            'x_merc':[to_mercator(42.0267,-93.6465)[0],
                      to_mercator(41.9987,-93.6223)[0],
                      to_mercator(42.0494,-93.6224)[0],
                      to_mercator(42.0323,-93.6111)[0],
                      to_mercator(42.0140,-93.6359)[0]],
            'y_merc':[to_mercator(42.0267,-93.6465)[1],
                      to_mercator(41.9987,-93.6223)[1],
                      to_mercator(42.0494,-93.6224)[1],
                      to_mercator(42.0323,-93.6111)[1],
                      to_mercator(42.0140,-93.6359)[1]]}

marks = pd.DataFrame(landmarks)

In [8]:
# Set color mapping for House Prices
mycolors = linear_cmap(field_name='SalePrice', palette=Plasma10[::-1] ,low=min(map_data.SalePrice) ,high=max(map_data.SalePrice))

In [9]:
background = get_provider(CARTODBPOSITRON_RETINA) #CARTODBPOSITRON_RETINA, STAMEN_TONER
x_zoom = 7000
y_zoom = 5000

# Base Map Layer
fig = figure(plot_width=1200, plot_height=800,
             x_range=(Ames_center[0]-x_zoom, Ames_center[0]+y_zoom), 
             y_range=(Ames_center[1]-x_zoom, Ames_center[1]+y_zoom),
             x_axis_type="mercator", y_axis_type="mercator",
             title="Ames Iowa Housing Map")
fig.add_tile(background)


# Dots for Houses
my_hover = HoverTool(names=['House'])
my_hover.tooltips = [('Price', '@SalePrice')]
fig.circle(x="x_merc", y="y_merc",
           size=5,
           fill_color=mycolors, line_color=mycolors,
           fill_alpha=0.6,
           name='House',
           source=map_data)
fig.add_tools(my_hover)

# Big Dots for Landmarks, with Hover interactivity
my_hover = HoverTool(names=['landmark'])
my_hover.tooltips = [('X', '@landmarks')]
fig.circle(x="x_merc", y="y_merc",
           size=12,
           fill_color="dodgerblue", line_color='dodgerblue',
           fill_alpha=0.3,
           name='landmark',
           source=marks)
fig.add_tools(my_hover)

color_bar = ColorBar(color_mapper=mycolors['transform'], width=8,  location=(0,0),title="Price $(thousands)")

fig.add_layout(color_bar, 'right')



# Add draw tool
renderer = fig.multi_line([[1, 9]], [[5, 5]], line_width=4, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=4)
fig.add_tools(draw_tool)

show(fig)